# Treinamento de PPO com Ambiente QRMSA

Este notebook demonstra como configurar, treinar e monitorar um modelo PPO utilizando o ambiente `QRMSAEnvWrapper-v0` do pacote `optical_networking_gym`. Além disso, integraremos o TensorBoard para visualizar o progresso do treinamento em tempo real.

**Passos:**
1. Definir as modulações e a topologia.
2. Configurar o ambiente.
3. Criar wrappers para ajustes do espaço de observação.
4. Definir e treinar o modelo PPO.
5. Acompanhar o treinamento pelo TensorBoard.


# 1. Introdução

Instalação das bibliotecas necessárias
Execute apenas se as bibliotecas não estiverem instaladas



In [1]:
!pip install stable-baselines3 gymnasium optical-networking-gym tensorboard

# 2. Imports
Primeiro, assegure-se de que todas as bibliotecas necessárias estão instaladas. Caso contrário, execute a célula acima para instalá-las.

In [2]:
# Imports necessários
import random
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from typing import Tuple

# Stable Baselines3 para PPO
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env

# Imports específicos do optical_networking_gym
from optical_networking_gym.wrappers.qrmsa_gym import QRMSAEnvWrapper
from optical_networking_gym.topology import Modulation, get_topology


# 3. Definição das Modulações
Definimos as diferentes modulações que serão utilizadas no ambiente.

In [ ]:
def define_modulations() -> Tuple[Modulation, ...]:
    return (
       Modulation(
            name="BPSK",
            maximum_length=100_000,  # 100,000 km to ensure safety
            spectral_efficiency=1,
            minimum_osnr=12.6,
            inband_xt=-14,
        ),
        Modulation(
            name="QPSK",
            maximum_length=2_000,
            spectral_efficiency=2,
            minimum_osnr=12.6,
            inband_xt=-17,
        ),
        Modulation(
            name="8QAM",
            maximum_length=1_000,
            spectral_efficiency=3,
            minimum_osnr=18.6,
            inband_xt=-20,
        ),
        Modulation(
            name="16QAM",
            maximum_length=500,
            spectral_efficiency=4,
            minimum_osnr=22.4,
            inband_xt=-23,
        ),
        Modulation(
            name="32QAM",
            maximum_length=250,
            spectral_efficiency=5,
            minimum_osnr=26.4,
            inband_xt=-26,
        ),
        Modulation(
            name="64QAM",
            maximum_length=125,
            spectral_efficiency=6,
            minimum_osnr=30.4,
            inband_xt=-29,
        ),
    )

# Definição das modulações atuais
cur_modulations = define_modulations()


# 4. Carregamento e Configuração da Topologia
Carregamos a topologia da rede óptica a partir de um arquivo de configuração.

In [4]:
# Nome e caminho da topologia
topology_name = "ring_4"

topology_path = (
    rf"C:\Users\talle\Documents\Mestrado\optical-networking-gym\examples\topologies\{topology_name}.txt"
    # Ajuste o caminho conforme necessário
)

# Carregamento da topologia
topology = get_topology(
    topology_path,
    "Ring4",          # Nome da topologia
    cur_modulations,  # Modulações
    80,               # Comprimento máximo do span em km
    0.2,              # Atenuação padrão em dB/km
    4.5,              # Figura de ruído padrão em dB
    5                 # Número de caminhos mais curtos a serem computados entre pares de nós
)


# 5. Definição dos Parâmetros do Ambiente
Definimos todos os parâmetros necessários para configurar o ambiente de simulação.

In [ ]:
# Parâmetros de simulação
seed = 10
random.seed(seed)

episode_length = 50000
load = 210
threads = 1
launch_power = 0

# Parâmetros do espectro
num_slots = 160
frequency_slot_bandwidth = 12.5e9  # 12.5 GHz
frequency_start = 3e8 / 1565e-9  
bandwidth = num_slots * frequency_slot_bandwidth
frequency_end = frequency_start + bandwidth
bit_rates = (10, 40, 80, 100, 400)  # Bit rates em Gbps
margin = 0

# Argumentos para o ambiente
env_args = dict(
    topology=topology,
    seed=seed,
    allow_rejection=True,
    load=load,
    episode_length=episode_length,
    num_spectrum_resources=num_slots,
    launch_power_dbm=launch_power,
    bandwidth=bandwidth,
    frequency_start=frequency_start,
    frequency_slot_bandwidth=frequency_slot_bandwidth,
    bit_rate_selection="discrete",
    bit_rates=bit_rates,
    margin=margin,
    file_name=f"./results/PPO_{1}",
    measure_disruptions=False,
    k_paths=2,
)


# 6. Criação e Verificação do Ambiente
Criamos o ambiente de simulação e verificamos sua conformidade utilizando o check_env do Stable Baselines3.

In [6]:
# ID do ambiente registrado no Gym
env_id = 'QRMSAEnvWrapper-v0'

# Criação do ambiente
env = gym.make(env_id, **env_args)

# Verificação do ambiente
check_env(env)


c:\Users\talle\Documents\Mestrado\optical-networking-gym\.venv\Lib\site-packages\stable_baselines3\common\env_checker.py:62: UserWarning: It seems that your observation space mask is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
c:\Users\talle\Documents\Mestrado\optical-networking-gym\.venv\Lib\site-packages\stable_baselines3\common\env_checker.py:75: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom features extractor cf. https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  warnings.warn(


# 7. Wrapper de Observação
Wrapper que extrai apenas a parte da observação necessária para o modelo, simplificando a entrada.

In [7]:
class ObservationOnlyWrapper(gym.Wrapper):
    def __init__(self, env):
        super(ObservationOnlyWrapper, self).__init__(env)
        reset_obs, _ = env.reset()
        self.observation_space = spaces.Box(
            low=np.float32(np.min(reset_obs['observation'])),
            high=np.float32(np.max(reset_obs['observation'])),
            shape=reset_obs['observation'].shape,
            dtype=np.float32
        )

    def reset(self, **kwargs):
        obs, info = self.env.reset()
        return obs['observation'], info

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        return obs['observation'], reward, terminated, truncated, info


# 8. Criação do Ambiente Vetorizado
Vetorizamos o ambiente para compatibilidade com os algoritmos do Stable Baselines3. Neste caso, utilizamos apenas um ambiente, mas é possível aumentar o número de ambientes paralelos ajustando n_envs.

In [8]:
def make_env():
    env = gym.make(env_id, **env_args)
    env = ObservationOnlyWrapper(env)
    return env

# Criação de um ambiente vetorizado com 1 ambiente
vec_env = make_vec_env(make_env, n_envs=1)


# 9. Definição e Treinamento do Modelo PPO
Configuramos e treinamos o modelo PPO, especificando os hiperparâmetros e o diretório para os logs do TensorBoard.

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback

def linear_schedule(initial_value):
    return lambda progress_remaining: progress_remaining * initial_value

eval_callback = EvalCallback(
    vec_env,
    best_model_save_path="./logs/best_model",
    log_path="./logs/",
    eval_freq=10000,
    deterministic=True,
    render=False,
)

model = PPO(
    "MlpPolicy",
    vec_env,
    learning_rate=linear_schedule(3e-4),  # Ajuste dinâmico da taxa de aprendizado
    n_steps=4096,  # Passos maiores para mais dados
    batch_size=256,  # Maior batch para estabilidade
    gamma=0.95,  # Desconto mais curto para recompensas
    gae_lambda=0.95,  # Igual ao valor original
    clip_range=0.1,  # Clipping mais conservador
    ent_coef=0.01,  # Incentivo para explorar mais
    seed=42,  # Reprodutibilidade
    tensorboard_log="./ppo_qrmsa_tensorboard/",
    verbose=1,
)

model.learn(total_timesteps=1_000_000, callback=eval_callback)
model.save("ppo_qrmsa_model")


In [17]:

modelo_carregado = PPO.load("ppo_qrmsa_model", env=vec_env)

obs = vec_env.reset()
for _ in range(600):
    action, _states = modelo_carregado.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)

In [10]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback, BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv

def linear_schedule(initial_value):
    """
    Retorna uma função (lambda) que, dado o `progress_remaining` (entre 1 e 0),
    retorna um valor linearly decaído. Usado no learning_rate.
    """
    return lambda progress_remaining: progress_remaining * initial_value

class EntropyCoefficientScheduler(BaseCallback):
    """
    Callback para atualizar dinamicamente o ent_coef do PPO.
    Vai reduzir linearmente de um valor inicial até um valor final ao longo de todo o treinamento.
    """
    def __init__(self, initial_ent_coef, final_ent_coef, schedule_timesteps, verbose=0):
        super(EntropyCoefficientScheduler, self).__init__(verbose)
        self.initial_ent_coef = initial_ent_coef
        self.final_ent_coef = final_ent_coef
        self.schedule_timesteps = schedule_timesteps

    def _on_step(self) -> bool:
        # Progresso do treinamento (0 a 1)
        progress = self.model.num_timesteps / self.schedule_timesteps
        if progress > 1.0:
            progress = 1.0
        
        # ent_coef linear entre initial e final
        current_ent_coef = self.initial_ent_coef + (self.final_ent_coef - self.initial_ent_coef) * progress
        self.model.ent_coef = current_ent_coef
        return True


eval_callback = EvalCallback(
    vec_env,
    best_model_save_path="./logs/best_model",
    log_path="./logs/",
    eval_freq=10000,
    deterministic=True,
    render=False,
)

# Cria o callback para agendamento do ent_coef
ent_scheduler = EntropyCoefficientScheduler(
    initial_ent_coef=0.07,  # Valor inicial de entropia
    final_ent_coef=0.1,     # Entropia final (menor, incentivando mais exploração no início e menos no fim)
    schedule_timesteps=1_000_000
)

model = PPO(
    "MlpPolicy",
    vec_env,
    learning_rate=linear_schedule(3e-4),  # Aprendizado dinâmico
    n_steps=2048,        # Menos passos por atualização (anteriormente 4096)
    batch_size=128,      # Batch menor (anteriormente 256)
    gamma=0.99,          # Maior valor de desconto (anteriormente 0.95)
    gae_lambda=0.95,
    clip_range=0.2,      # Ligeiramente menos conservador (antes 0.1)
    ent_coef=0.01,       # Valor inicial do ent_coef, será sobrescrito pelo callback
    seed=42,
    tensorboard_log="./ppo_qrmsa_tensorboard/",
    verbose=1,
    policy_kwargs=dict(net_arch=[256, 256])  # Rede maior para melhor capacidade de representação
)

model.learn(total_timesteps=1_000_000, callback=[eval_callback, ent_scheduler])
model.save("ppo_qrmsa_model_dynamic_ent")


Using cpu device
Logging to ./ppo_qrmsa_tensorboard/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 598      |
|    ep_rew_mean     | 53.7     |
| time/              |          |
|    fps             | 224      |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 598         |
|    ep_rew_mean          | 55.8        |
| time/                   |             |
|    fps                  | 203         |
|    iterations           | 2           |
|    time_elapsed         | 20          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.028549545 |
|    clip_fraction        | 0.367       |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.07       |
|    explained

In [23]:
info

[{'episode_services_accepted': 2,
  'service_blocking_rate': 0,
  'episode_service_blocking_rate': 0,
  'bit_rate_blocking_rate': 0.0,
  'episode_bit_rate_blocking_rate': 0.5555555555555556,
  'disrupted_services': 0.0,
  'episode_disrupted_services': 0.0,
  'modulation_1': 0,
  'modulation_2': 2,
  'modulation_4': 0,
  'modulation_6': 0,
  'TimeLimit.truncated': False}]